In [1]:
import hashlib
import os
from typing import Tuple, Dict, Optional


class ECDSA:

    def __init__(self):
        # Tham số đường cong elliptic secp256k1
        # Phương trình: y² ≡ x³ + 7 (mod p)
        
        # Số nguyên tố p xác định trường hữu hạn Fp
        self.p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
        
        # Hệ số a và b của phương trình đường cong
        self.a = 0
        self.b = 7
        
        # Điểm cơ sở G (Base Point)
        self.G = {
            'x': 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
            'y': 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8
        }
        # Bậc của nhóm (số lượng điểm trên đường cong)
        self.n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
        
        # Cofactor (thường là 1)
        self.h = 1

    def mod_inverse(self, a: int, m: int) -> int:
        """
        Tính nghịch đảo modular của a modulo m
        Sử dụng Extended Euclidean Algorithm
        """
        old_r, r = a, m
        old_s, s = 1, 0
        
        while r != 0:
            quotient = old_r // r
            old_r, r = r, old_r - quotient * r
            old_s, s = s, old_s - quotient * s
        
        return old_s if old_s >= 0 else old_s + m

    def point_add(self, p1: Optional[Dict], p2: Optional[Dict]) -> Optional[Dict]:
        """
        Cộng hai điểm trên đường cong elliptic
    
        """
        # Nếu một trong hai điểm là điểm vô cùng, trả về điểm còn lại
        if p1 is None:
            return p2
        if p2 is None:
            return p1

        x1, y1 = p1['x'], p1['y']
        x2, y2 = p2['x'], p2['y']

        if x1 == x2:
            if y1 == y2:
                # Trường hợp điểm nhân đôi (point doubling)
                # Tính độ dốc: s = (3*x1²) / (2*y1)
                s = (3 * x1 * x1) * self.mod_inverse(2 * y1, self.p)
                s = s % self.p
                
                # Tính tọa độ điểm mới
                x3 = (s * s - 2 * x1) % self.p
                y3 = (s * (x1 - x3) - y1) % self.p
                
                return {'x': x3, 'y': y3}
            else:
                # Hai điểm có cùng x nhưng y khác nhau => tổng là điểm vô cùng
                return None

        # Trường hợp bình thường: hai điểm khác nhau
        # Tính độ dốc: s = (y2 - y1) / (x2 - x1)
        s = ((y2 - y1) * self.mod_inverse(x2 - x1, self.p)) % self.p
        
        # Tính tọa độ điểm mới
        x3 = (s * s - x1 - x2) % self.p
        y3 = (s * (x1 - x3) - y1) % self.p

        return {'x': x3, 'y': y3}

    def point_multiply(self, k: int, point: Dict) -> Optional[Dict]:
        """
        Nhân một điểm với một vô hướng (Scalar Multiplication)
        Sử dụng Binary Method (Double-and-Add)
        
        Tính Q = k * P một cách hiệu quả
        """
        if k == 0:
            return None
        if k == 1:
            return point

        result = None
        addend = point

        while k:
            # Nếu bit thấp nhất của k là 1, cộng addend vào result
            if k & 1:
                result = self.point_add(result, addend)
            
            # Nhân đôi addend, chia k cho 2
            addend = self.point_add(addend, addend)
            k >>= 1

        return result

    def generate_key_pair(self) -> Dict:
        """
        Tạo một cặp khóa ECDSA
        Returns:
            Dict chứa 'privateKey' và 'publicKey'
        """
        # Khóa riêng: số ngẫu nhiên d trong khoảng [1, n-1]
        d = int.from_bytes(os.urandom(32), 'big') % self.n
        if d == 0:
            d = 1

        # Khóa công khai: điểm Q = d * G
        Q = self.point_multiply(d, self.G)

        return {
            'privateKey': d,
            'publicKey': Q
        }

    @staticmethod
    def sha256(message: str) -> str:
        """
        Tính SHA-256 hash của một thông điệp
    
        """
        hash_obj = hashlib.sha256(message.encode('utf-8'))
        return hash_obj.hexdigest()

    def sign(self, message: str, private_key: int) -> Dict:
        """
        Ký một thông điệp bằng khóa riêng
        
        Quá trình ký:
        1. Hash thông điệp bằng SHA-256: z = H(m)
        2. Chọn số ngẫu nhiên k từ [1, n-1]
        3. Tính điểm R = k*G, lấy r = R.x mod n
        4. Tính s = k⁻¹(z + r*d) mod n
        5. Chữ ký là (r, s)
        """
        # Bước 1: Hash thông điệp
        h = self.sha256(message)
        z = int(h, 16) % self.n

        # Bước 2: Chọn số ngẫu nhiên k
        while True:
            k = int.from_bytes(os.urandom(32), 'big') % self.n
            if k != 0:
                break

        # Bước 3: Tính R = k*G
        R = self.point_multiply(k, self.G)
        r = R['x'] % self.n

        if r == 0:
            # Nếu r = 0, chọn k khác và ký lại
            return self.sign(message, private_key)

        # Bước 4: Tính s = k⁻¹(z + r*d) mod n
        s = (self.mod_inverse(k, self.n) * (z + r * private_key)) % self.n

        if s == 0:
            # Nếu s = 0, chọn k khác và ký lại
            return self.sign(message, private_key)

        return {'r': r, 's': s, 'hash': h}

    def verify(self, message: str, public_key: Dict, signature: Dict) -> bool:
        """
        Xác thực một chữ ký
        
        Quá trình xác thực:
        1. Hash thông điệp: z = H(m)
        2. Tính w = s⁻¹ mod n
        3. Tính u1 = z*w mod n, u2 = r*w mod n
        4. Tính P = u1*G + u2*Q
        5. Nếu P.x mod n == r, chữ ký hợp lệ
        """
        r, s = signature['r'], signature['s']

        # Kiểm tra r và s nằm trong khoảng hợp lệ
        if not (1 <= r < self.n and 1 <= s < self.n):
            return False

        # Bước 1: Hash thông điệp
        h = self.sha256(message)
        z = int(h, 16) % self.n

        # Bước 2: Tính w = s⁻¹ mod n
        w = self.mod_inverse(s, self.n)

        # Bước 3: Tính u1 và u2
        u1 = (z * w) % self.n
        u2 = (r * w) % self.n

        # Bước 4: Tính P = u1*G + u2*Q
        P1 = self.point_multiply(u1, self.G)
        P2 = self.point_multiply(u2, public_key)
        P = self.point_add(P1, P2)

        if P is None:
            return False

        # Bước 5: Kiểm tra P.x mod n == r
        return (P['x'] % self.n) == r

if __name__ == '__main__':
    # Khởi tạo ECDSA
    ecdsa = ECDSA()

    # 1. Tạo cặp khóa
    print("=" * 70)
    print("1. TẠO CẶP KHÓA")
    print("=" * 70)
    keypair = ecdsa.generate_key_pair()
    private_key = keypair['privateKey']
    public_key = keypair['publicKey']

    print(f"Khóa riêng (Private Key): {hex(private_key)}")
    print(f"Khóa công khai (Public Key):")
    print(f"  X: {hex(public_key['x'])}")
    print(f"  Y: {hex(public_key['y'])}")

    # 2. Ký một thông điệp
    print("\n" + "=" * 70)
    print("2. KÝ THÔNG ĐIỆP")
    print("=" * 70)
    message = "Hello, ECDSA secp256k1!"
    signature = ecdsa.sign(message, private_key)

    print(f"Thông điệp: {message}")
    print(f"SHA-256 Hash: {signature['hash']}")
    print(f"Chữ ký r: {hex(signature['r'])}")
    print(f"Chữ ký s: {hex(signature['s'])}")

    # 3. Xác thực chữ ký
    print("\n" + "=" * 70)
    print("3. XÁC THỰC CHỮ KÝ")
    print("=" * 70)
    is_valid = ecdsa.verify(message, public_key, signature)
    print(f"Kết quả xác thực: {'✓ Hợp lệ' if is_valid else '✗ Không hợp lệ'}")

    # 4. Thử xác thực với thông điệp sai
    print("\n" + "=" * 70)
    print("4. XÁC THỰC VỚI THÔNG ĐIỆP SAI")
    print("=" * 70)
    wrong_message = "Hello, Wrong Message!"
    is_valid_wrong = ecdsa.verify(wrong_message, public_key, signature)
    print(f"Thông điệp sai: {wrong_message}")
    print(f"Kết quả xác thực: {'✓ Hợp lệ' if is_valid_wrong else '✗ Không hợp lệ'}")

1. TẠO CẶP KHÓA
Khóa riêng (Private Key): 0x2c83fea991ec233a2957337b568457c5232937488769f979392234d37678da2b
Khóa công khai (Public Key):
  X: 0xb4b5fe23b8be4da6cba36a017ded6f3e14d04452e97947142b0860b125e6ec6
  Y: 0x21b1952d2057da101a7708ca7211c02e5209da8205023cd72db03c9389ebc8e4

2. KÝ THÔNG ĐIỆP
Thông điệp: Hello, ECDSA secp256k1!
SHA-256 Hash: 0943eaa94ccbe8896daa10fc713a23e9fbbf2e7696986fc40891e4a6cf643af3
Chữ ký r: 0x654d14e08c00ee875b1437597dd8b9450b85d16201b0f42d3ba3cb6d006eb1aa
Chữ ký s: 0x305d3d4e2175dada88e9fab4b46514ac51c2807dfeb8ff55818d2a142b52a49

3. XÁC THỰC CHỮ KÝ
Kết quả xác thực: ✓ Hợp lệ

4. XÁC THỰC VỚI THÔNG ĐIỆP SAI
Thông điệp sai: Hello, Wrong Message!
Kết quả xác thực: ✗ Không hợp lệ
